В предыдущих уроках вы изучили BatchNorm и Dropout. Теперь пора посмотреть, как они работают.

Поупражняйтесь на датасете рукописных цифр MNIST и убедитесь, что даже на одной эпохе BatchNorm улучшает качество модели. 

В этой задаче вы можете менять только класс модели. Исходный код дает точность ≈96.31% на тесте. С помощью BatchNorm можно её повысить до значения более 97%. 

В этой задаче нет одного правильного решения, вам нужно эксперементировать.

# Задача

Запустите исходный код, чтобы получить базовые результаты обучения модели (англ. *baseline*). Затем добавьте один или несколько слоёв BatchNorm (пример `nn.BatchNorm1d(num_inputs)`) в модель `NeuralNet` и сравните итоги работы модели с использованием этого метода и без.

Кроме того, вы можете добавить слой Dropout (пример `nn.Dropout(p=0.5)`) и посмотреть, как он сочетается с BatchNorm. Так вы сможете понять, почему два этих метода не рекомендуется использовать одновременно.

Установите необходимые пакеты. Возможно, после установки потребуется перезапустить ядро Jupyter через пункт меню Kernel -> Restart. 

In [ ]:
!pip install torch==1.10.0+cpu torchvision==0.11.0+cpu torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html --user


Импортируйте необходимые пакеты

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

Гиперпараметры обучения:

In [ ]:

torch.manual_seed(1234)
input_size = 28*28
hidden_size = 800
num_classes = 10
num_epochs = 1
batch_size = 100
learning_rate = 0.001


Создайте датасет и даталоадер:

In [ ]:
# MNIST датасет
train_dataset = torchvision.datasets.MNIST(root='data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Dataloader для тренировочной и тестовой части датасета 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


Создайте модель

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

Обучение и проверка точности на тесте:

In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()  # В качестве функции потерь используем Кросс-Энтропию
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # создаем Adam оптимизатор

# Цикл обучения
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, input_size)
        
        # Прямой проход
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Обратный проход
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        if (i+1) % 100 == 0:
            print ('Эпоха [{}/{}], Итерация [{}/{}], Значение функции потерь: {:.4f} Точность на батче: {}%' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(), 100 *(predicted == labels).sum()/labels.size(0)))

# Считаем точность на тестовой выборке
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        labels = labels
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Точность на тесте: {} %'.format(100 * correct / total))